In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report, multilabel_confusion_matrix

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

import nltk
import tensorflow as tf

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.naive_bayes import MultinomialNB
from keras.layers import Dense, Input, LSTM, GRU, Conv1D, Dropout, Flatten, Layer, BatchNormalization
import string
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from tensorflow import keras

from gensim.models import KeyedVectors
from gensim import models

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Data
df = pd.read_csv('mbti_1.csv')
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


### MBTI exsit in 4 different Categories having Two classes, we will be sepearting into Four Binary Classification

First Section is E and I seperation (First Section of MBTI)

In [3]:
# Seperating Type of MBTI to Binary (E and I)
def binary_seperation(dataset, mbti_type1, mbti_type2):
    for i in range(len(df['type'])):
        if mbti_type1 in dataset['type'].iloc[i]:
            dataset['type'].iloc[i] = mbti_type1
        else:
            dataset['type'].iloc[i] = mbti_type2
    return dataset

In [4]:
ei = binary_seperation(df, 'E', 'I')
ei.head()

,type,posts
0,I,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,E,'I'm finding the lack of me in these posts ver...
2,I,'Good one _____ https://www.youtube.com/wat...
3,I,"'Dear INTP, I enjoyed our conversation the o..."
4,E,'You're fired.|||That's another silly misconce...


When doing NLP, the Website link does not give inutitive information when predicting, we will be removing the Website links to each posts

In [5]:
# Removing Text with website link and expanding each user's posts
# one row = one user having 50 different posts --> 1 Row with one post with corresponding MBTI Type
intravert = []
extravert = []
for i in range(len(ei)):
    if df['type'].iloc[i] == 'I':
        split = df['posts'].iloc[i].split('|||')
        for sentence in split:
            if 'http' not in sentence:
                intravert.append(sentence)
                
    else:
        split = df['posts'].iloc[i].split('|||')
        for sentence in split:
            if 'http' not in sentence:
                extravert.append(sentence)
    
I = pd.DataFrame(intravert, columns = ['Posts'])
I['type'] = 'I'

E = pd.DataFrame(extravert, columns = ['Posts'])
E['type'] = 'E'

eiei = pd.concat([I, E])

In [7]:
eiei.head()

,Posts,type
0,What has been the most life-changing experienc...,I
1,May the PerC Experience immerse you.,I
2,Hello ENFJ7. Sorry to hear of your distress. I...,I
3,Welcome and stuff.,I
4,"Prozac, wellbrutin, at least thirty minutes of...",I


Before Feature Extraction, the text must be preprocessed:
1. Removing Stop Words
2. Removing Punctuations
3. Apply Stemming
4. Apply Lemmatization

In [8]:
# Function that Takes in the Text (in sentence) and remove unecessary portion from the text
def text_preprocess(headlines):
    #Tokenizing Each headlines 
    token = RegexpTokenizer(r'[a-zA-Z0-9]+')

    tokenized = []
    
    #Tokenizing + Number Removal
    for i in range(len(headlines)):
        tokens = token.tokenize(headlines.iloc[i])
        num_remove = [i for i in tokens if i.isalpha()]
        tokenized.append(num_remove)
        
    #Stop words and Punctuation removal
    st = stopwords.words('english')
    punctuations = string.punctuation
    processed = []
    
    for i in range(len(tokenized)):
        words = [word for word in tokenized[i] if word.lower() not in st and word not in punctuations]
        #Num Removal
        new_text = " ".join(words)
        processed.append(new_text)
    
    #Stemming
    stemmer = PorterStemmer()
    stemmed = []
    
    for i in processed:
        stem_word = stemmer.stem(i)
        stemmed.append(stem_word)
    
    #Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized = []
    
    for i in stemmed:
        lemmatize_word = lemmatizer.lemmatize(i)
        lemmatized.append(lemmatize_word)
      
    return lemmatized

In [9]:
preprocessed = text_preprocess(eiei['Posts'])
eiei['Posts'] = preprocessed
eiei.reset_index(inplace = True, drop=True)
eiei.head()

,Posts,type
0,life changing experience lif,I
1,may perc experience immers,I
2,hello sorry hear distress natural relationship...,I
3,welcome stuff,I
4,prozac wellbrutin least thirty minutes moving ...,I


When text sizes are too Small Ex. Less than 2, it will not provide a enough information when predicting, removing the posts with less than 2 words will be removed

In [10]:
#Removing the sentence with less than or equal to 2 words (this will not provide inutitive information)
drop = []
for i in range(len(eiei['Posts'])):
    if len(eiei['Posts'].iloc[i].split()) <= 2:
        drop.append(i)

eiei.drop(drop, inplace = True)

### The data had ~8700 Rows (Sample Size) but each row contained 50 different posts within a single row, I have decided to expand this to make more datasets

In [12]:
eiei

,Posts,type
0,life changing experience lif,I
1,may perc experience immers,I
2,hello sorry hear distress natural relationship...,I
4,prozac wellbrutin least thirty minutes moving ...,I
5,basically come three items determined type whi...,I
...,...,...
397609,knit craft long craft simple complicated patte...,E
397610,quote kevinaswell say green grape soda say huh...,E
397611,true sort hahah goes cycles spurts interest,E
397612,dear anonymous es hire great asset possible fu...,E
